In [1]:
from platform import python_version
python_version()

import os
import pandas as pd
import csv

print(python_version())

3.9.12


In [2]:
### this program gives top 50 deliveries by value and volume for fno and non-fno. 
#### it also creates fno_csv and non_fno_csv which is used by multiple scrips program

### caution - if data file is already read, then it skips reading it. So if data is updated, close the program and restart it
#### to enable data file to be read again


In [3]:
### lenovo desktop file path
master_file_path = "C:\\Users\\User\\Dropbox\\SQL Stock Market Data\\Python Code from ChatGPT\\Master File\\CompanyMaster.csv"
directory = "C:\\Users\\User\\Dropbox\\SQL Stock Market Data\\Data Files\\"  # Replace with the path to your directory
output_path = "C:\\Users\\User\\Dropbox\\SQL Stock Market Data\\Python Code from ChatGPT\\Output File\\"

column_order = ['NSE_Code','BSE_Code','Date','NSE_Delivery','BSE_Delivery']





In [4]:
#### Lenovo laptop file path

master_file_path = "C:\\Users\\sarda\\Dropbox\\SQL Stock Market Data\\Python Code from ChatGPT\\Master File\\CompanyMaster.csv"
directory = "C:\\Users\\sarda\\Dropbox\\SQL Stock Market Data\\Data Files\\"  # Replace with the path to your directory
output_path = "C:\\Users\\sarda\\Dropbox\\SQL Stock Market Data\\Python Code from ChatGPT\\Output File\\"

column_order = ['NSE_Code','BSE_Code','Date','NSE_Delivery','BSE_Delivery']


#### datapath for lenovo laptop

master_file_path = "C:\\Users\\sarda\\Dropbox\\SQL Stock Market Data\\Python Code from ChatGPT\\Master File\\CompanyMaster.csv"
directory = "C:\\Users\\sarda\\Dropbox\\SQL Stock Market Data\\Data Files\\"  # Replace with the path to your directory

output_path = "C:\\Users\\sarda\\Dropbox\\SQL Stock Market Data\\Python Code from ChatGPT\\Output File\\"
column_order = ['NSE_Code','BSE_Code','Date','NSE_Delivery','BSE_Delivery']
file_names_start_with = ['NSE Delivery','BSE Gross Deliverable', 'NSE EQ', 'EQ ISIN']
file_names = ['nse_del_files','bse_del_files','nse_price_files','bse_price_files']

non_fno_csv = output_path + '0_non_fno.csv'
fno_csv = output_path + 'fno.csv'
sme_csv = output_path +'sme_csv.csv'

In [5]:


def nse_data(nse_file_name):
    nse_df = pd.DataFrame(columns=['NSE_Code', 'Date','NSE_Volume', 'NSE_Delivery', 'Segment'])
    dely_segment = ['EQ', 'BE', 'SM', 'ST']
    data_to_append = []

    for file_name in nse_file_name:
        file_path = os.path.join(directory, file_name)
        date = file_name[13:19]

        with open(file_path, 'r') as file:
            reader = csv.reader(file)
            for i, line in enumerate(reader, start=1):
                if i >= 5:
                    line_str = ','.join(line)
                    data = line_str.split(',')
                    if data[3] in dely_segment:
                        nse_code = data[2]
                        volume = data[4]
                        dely = data[5]
                        segment = data[3]
                        data_to_append.append([nse_code, date, volume,dely, segment])

    if data_to_append:
        nse_df = pd.concat([nse_df, pd.DataFrame(data_to_append, columns=['NSE_Code', 'Date', 'NSE_Volume','NSE_Delivery', 'Segment'])], ignore_index=True)
        nse_df['NSE_Volume'] = nse_df['NSE_Volume'].astype(int)
        nse_df['NSE_Delivery'] = nse_df['NSE_Delivery'].astype(int)

    return nse_df



def nse_price(nse_price_file):
    nse_df = pd.DataFrame(columns=['NSE_Code', 'Date', 'Traded', 'Price', 'PrevClose', 'Segment'])
    dely_segment = ['EQ', 'BE', 'SM', 'ST']
    data_to_append = []

    for file_name in nse_price_file:
        file_path = os.path.join(directory, file_name)
        date = file_name[7:13]

        with open(file_path, 'r') as file:
            reader = csv.reader(file)
            for row in reader:
                if row[1] in dely_segment:
                    nse_code = row[0]
                    price = row[5]
                    prev_close = row[7]
                    segment = row[1]
                    traded = row[8]
                    data_to_append.append([nse_code, date, price, prev_close, segment, traded])

    if data_to_append:
        nse_df = pd.concat([nse_df, pd.DataFrame(data_to_append, columns=['NSE_Code', 'Date', 'Price', 'PrevClose', 'Segment', 'Traded'])], ignore_index=True)
        nse_df['Traded'] = nse_df['Traded'].astype(int)
        nse_df['Price'] = nse_df['Price'].astype(float)
        nse_df['PrevClose'] = nse_df['PrevClose'].astype(float)
        

    return nse_df


def bse_data(bse_file_name):
    # Create an empty DataFrame to store the results
    bse_df = pd.DataFrame(columns=['BSE_Code', 'Date', 'BSE_Volume', 'BSE_Delivery'])
    dely_segment = ['EQ', 'BE', 'SM', 'ST']
    data_to_append = []

    for file_name in bse_file_name:
        file_path = os.path.join(directory, file_name)
        date = file_name[22:28]

        with open(file_path, 'r') as file:
            reader = csv.reader(file)
            for i, line in enumerate(reader, start=1):
                if i >= 2:  # Start reading from line 5
                    line_str = ','.join(line)
                    data = line_str.split('|')
                    volume = data[4]
                    dely = data[2]
                    bse_code = data[1]
                    data_to_append.append([bse_code, date, volume, dely])

    if data_to_append:
        bse_df = pd.concat([bse_df, pd.DataFrame(data_to_append, 
                                                 columns=['BSE_Code', 'Date', 'BSE_Volume', 'BSE_Delivery'])])
        
        bse_df['BSE_Volume'] = bse_df['BSE_Volume'].astype(int)
        bse_df['BSE_Delivery'] = bse_df['BSE_Delivery'].astype(int)
    
        
        bse_df = bse_df.drop(0).reset_index(drop=True)

    return bse_df

def bse_price(bse_price_file):
# Create an empty DataFrame to store the results
    
    
    bse_df = pd.DataFrame(columns=['BSE_Code', 'Date','BSE_Volume','Price','PrevClose','Segment'])
    dely_segment = ['EQ','BE','SM','ST']
    data_to_append = []

    

    for file_name in bse_price_file:
        file_path = os.path.join(directory, file_name)
        date = file_name[8:14]
            
        with open(file_path, 'r') as file:
            reader = csv.reader(file)
            for row in reader:
                    
                data_list = []
                data_list.append(row)
                    
                bse_code = data_list[0][0]
                bse_volume =  data_list[0][11]
                price = data_list[0][7]
                prev_close = data_list[0][9]
                bse_group = data_list[0][2]
                            
                data_to_append.append([bse_code, date, bse_volume, price, prev_close, bse_group])
                
                
    if data_to_append:
        bse_df = pd.concat([bse_df, pd.DataFrame(data_to_append, 
                                                 columns=['BSE_Code', 'Date', 'BSE_Volume', 'Price','PrevClose','Segment'])])
        
            
        bse_df = bse_df[bse_df['BSE_Code'] != 'SC_CODE']
        
        
        

    return bse_df


def nse_merge(dely_file,price_file):
    
    nse_dely_data = nse_data(dely_file)
    nse_price_data = nse_price(price_file)
    
    nse_merged_data = pd.merge(nse_dely_data, nse_price_data, on = ['NSE_Code','Date','Segment'], how='outer')
    
    nse_merged_data['NSE_Volume'].fillna(nse_merged_data['Traded'], inplace = True)
    nse_merged_data['NSE_Delivery'].fillna(nse_merged_data['Traded'], inplace = True)

    nse_merged_data['NSE_Volume'] = nse_merged_data['NSE_Volume'].astype(int)
    nse_merged_data['NSE_Delivery'] = nse_merged_data['NSE_Delivery'].astype(int)
    
    return nse_merged_data


def extract_date(file_name,start):
    
    for i in file_name:
        end = start + 6
        dely_date = i[start:end]
        
    return dely_date


### extract_date refers to i in file_name(list) whereas extract_date_2 is just file name

def extract_date_2(file_name, start):
    end = start + 6
    dely_date_2 = file_name[start:end]
    return dely_date_2


    
    



In [6]:


nse_dely_file = sorted([file for file in os.listdir(directory) if file.endswith('.csv') and file.startswith('NSE Delivery')], reverse=True)[:1]
bse_dely_file = sorted([file for file in os.listdir(directory) if file.endswith('.txt') and file.startswith('BSE Gross Deliverable ')], reverse=True)[:1]

nse_price_file = sorted([file for file in os.listdir(directory) if file.lower().endswith(('CSV','.csv')) and file.startswith('NSE EQ')], reverse=True)[:1]
bse_price_file = sorted([file for file in os.listdir(directory) if file.lower().endswith(('CSV','csv')) and file.startswith('EQ ISIN')], reverse=True)[:1]

nse_fno_file = sorted([file for file in os.listdir(directory) if file.startswith('NSE FNO')],reverse=True)[:1]



###result_str = str(int(number_str) + 6)
print(nse_dely_file)

nse_dely_date = extract_date(nse_dely_file,13)
bse_dely_date = extract_date(bse_dely_file,22)

nse_price_date = extract_date(nse_price_file,7)
bse_price_date = extract_date(bse_price_file,8)


if nse_dely_date != bse_dely_date != nse_price_date != bse_price_date:
    print("dates dont match")
    print(nse_dely_date, ' ' , bse_dely_date , ' ' , nse_price_date , ' ' , bse_price_date)
else:
    print("processing for :", nse_dely_date)


['NSE Delivery 231020.csv']
processing for : 231020


In [7]:



a = 0

for i in file_names_start_with:
    
    file_names[a] = sorted([file for file in os.listdir(directory) if file.startswith(i)], reverse = True)[:2]
    a = a + 1


nse_dely_files = file_names[0]
bse_dely_files = file_names[1]
nse_price_files = file_names[2]
bse_price_files = file_names[3]

print(nse_dely_files,nse_price_files,bse_dely_files,bse_price_files)

a = 0
nse_merge_df = []
for i in nse_dely_files:
    nse_dely_file = nse_dely_files[a]
    nse_dely_date = nse_dely_file[13:19]
    merge_file = 'nse_' + nse_dely_date
    nse_merge_df.append(merge_file)
    a = a + 1
       
print(nse_merge_df)  

['NSE Delivery 231020.csv', 'NSE Delivery 231019.csv'] ['NSE EQ 231020.csv', 'NSE EQ 231019.csv'] ['BSE Gross Deliverable 231020.txt', 'BSE Gross Deliverable 231019.txt'] ['EQ ISIN 231020.csv', 'EQ ISIN 231019.csv']
['nse_231020', 'nse_231019']


In [8]:
print(bse_dely_file, bse_price_file)
bse_dely_single = bse_data(bse_dely_file)
bse_price_single = bse_price(bse_price_file)
bse_dely_single, bse_price_single

['BSE Gross Deliverable 231020.txt'] ['EQ ISIN 231020.csv']


(     BSE_Code    Date  BSE_Volume  BSE_Delivery
 0      543150  231020       52954         52754
 1      543161  231020        1811          1409
 2      543167  231020        7814          7814
 3      543171  231020      245463        232046
 4      543173  231020          75            75
 ...       ...     ...         ...           ...
 4117   974519  231020           2             2
 4118   974520  231020           1             1
 4119   974561  231020           2             2
 4120   974749  231020           3             3
 4121   975057  231020           1             1
 
 [4122 rows x 4 columns],
      BSE_Code    Date BSE_Volume   Price PrevClose Segment
 1      500002  231020       3140  4015.7    4087.5      A 
 2      500003  231020      16301   313.7       318      A 
 3      500008  231020      59709   628.7    637.65      A 
 4      500009  231020     443534   41.45     42.27      XT
 5      500012  231020     116087   81.21     82.13      X 
 ...       ...     ...  

In [9]:
print(bse_dely_files, bse_price_files)
bse_dely_double = bse_data(bse_dely_files)


['BSE Gross Deliverable 231020.txt', 'BSE Gross Deliverable 231019.txt'] ['EQ ISIN 231020.csv', 'EQ ISIN 231019.csv']


In [10]:
bse_price_double = bse_price(bse_price_files)
bse_price_double


,BSE_Code,Date,BSE_Volume,Price,PrevClose,Segment
1,500002,231020,3140,4015.7,4087.5,A
2,500003,231020,16301,313.7,318,A
3,500008,231020,59709,628.7,637.65,A
4,500009,231020,443534,41.45,42.27,XT
5,500012,231020,116087,81.21,82.13,X
...,...,...,...,...,...,...
8255,975003,231019,1,102190,102109.59,F
8256,975059,231019,3,99500,0,F
8257,975087,231019,4,99635,99551,F
8258,975137,231019,5682,100441.01,100500,F


In [11]:
bse_price_double.sort_values(by='BSE_Volume', inplace = True)
bse_price_double.reset_index(drop=True, inplace=True)
bse_price_double


,BSE_Code,Date,BSE_Volume,Price,PrevClose,Segment
0,800434,231019,1,5930,5972.51,G
1,539495,231019,1,46,46,XT
2,502901,231020,1,4283,4079.75,X
3,531203,231020,1,38.71,40.74,X
4,539545,231020,1,21.5,21.43,X
...,...,...,...,...,...,...
8253,533270,231019,9984,135,137,T
8254,526899,231019,99842,21.14,21.69,X
8255,512329,231020,9990,7751.35,7599.4,XT
8256,537800,231020,999214,3.51,3.63,XT


In [12]:
# Count the records where the column has the target value
#count = len(df[df['column_name'] == target_value])

count = len(bse_price_double[bse_price_double['Date']  == '231020'])
#count = len(bse_price_double[bse_price_double['BSE_Code'] == 'SC_CODE'])
print(count)

4123


In [13]:
#bse_dely = bse_data(bse_dely_files)
#bse_dely

#bse_price = bse_price(bse_price_files)

bse_df = pd.DataFrame(columns=['BSE_Code', 'Date','BSE_Volume','Price','PrevClose','Segment'])
dely_segment = ['EQ','BE','SM','ST']
data_to_append = []

    

for file_name in bse_price_files:
    
    file_path = os.path.join(directory, file_name)
    date = file_name[8:14]
            
    with open(file_path, 'r') as file:
        
        reader = csv.reader(file)
        for row in reader:
            
            data_list = []
            data_list.append(row)
                    
            bse_code = data_list[0][0]
            bse_volume =  data_list[0][11]
            price = data_list[0][7]
            prev_close = data_list[0][9]
            bse_group = data_list[0][2]
            data_to_append.append([bse_code, date, bse_volume, price, prev_close, bse_group])
                

In [14]:
bse_dely = bse_data(bse_dely_files)
bse_price = bse_price(bse_price_files)

#bse_price['BSE_Volume'] = bse_price['BSE_Volume'].astype(int)
#bse_price['Price'] = bse_price['Price'].astype(float)
#bse_price['PrevClose'] = bse_price['PrevClose'].astype(float)

#bse_merge = pd.merge(bse_dely, bse_price, on=['BSE_Code','Date','BSE_Volume'], how='outer')

#bse_merge['BSE_Delivery'].fillna(0, inplace = True)
#bse_merge['BSE_Delivery'] = bse_merged_data['BSE_Delivery'].astype(int)
#bse_merge.reset_index(inplace=True, drop=True)

#bse_merge
bse_dely
bse_price

,BSE_Code,Date,BSE_Volume,Price,PrevClose,Segment
1,500002,231020,3140,4015.7,4087.5,A
2,500003,231020,16301,313.7,318,A
3,500008,231020,59709,628.7,637.65,A
4,500009,231020,443534,41.45,42.27,XT
5,500012,231020,116087,81.21,82.13,X
...,...,...,...,...,...,...
8255,975003,231019,1,102190,102109.59,F
8256,975059,231019,3,99500,0,F
8257,975087,231019,4,99635,99551,F
8258,975137,231019,5682,100441.01,100500,F


In [15]:
bse_price = bse_price(bse_price_files)


TypeError: 'DataFrame' object is not callable

In [ ]:
bse_dely_data = bse_data(bse_dely_file)

#bse_dely_data['BSE_Volume'] = bse_dely_data['BSE_Volume'].astype(int)
#bse_dely_data['BSE_Delivery'] = bse_dely_data['BSE_Delivery'].astype(int)

bse_price_data = bse_price(bse_price_file)

bse_price_data['BSE_Volume'] = bse_price_data['BSE_Volume'].astype(int)
bse_price_data['Price'] = bse_price_data['Price'].astype(float)
bse_price_data['PrevClose'] = bse_price_data['PrevClose'].astype(float)


bse_merged_data = pd.merge(bse_dely_data, bse_price_data, on=['BSE_Code','Date','BSE_Volume'], how='outer')

bse_merged_data['BSE_Delivery'].fillna(0, inplace = True)
bse_merged_data['BSE_Delivery'] = bse_merged_data['BSE_Delivery'].astype(int)
bse_merged_data.reset_index(inplace=True, drop=True)


#bse_merged_data.info()
#print()
#print(bse_merged_data)

In [ ]:
bse_merged_data.info()
bse_merged_data

In [ ]:
#nse_merged_data
#bse_merged_data


In [ ]:
### getting unique symbols from fno

for file_name in nse_fno_file:
    file_path = os.path.join(directory, file_name)
    fno_df = pd.read_csv(file_path)
    fno_df = fno_df.drop_duplicates(subset ='SYMBOL')
    fno_symbols = fno_df['SYMBOL']
    fno_symbols = fno_symbols.reset_index(drop=True)

In [ ]:
## reading master file
### master file (csv) is Company Name	CD_NSE Symbol	CD_BSE Code	CD_Is Subsidiary	CD_Sector	CD_Industry1

#### TO FIX - MASTER FILE DOES NOT HAVE SM SEGMENT CODES

master_file = pd.read_csv(master_file_path, dtype={'CD_BSE Code': str})


###df.rename(columns={'old_column_name': 'new_column_name'}, inplace=True)
##df.rename(columns={'old_column_name': 'new_column_name'}, inplace=True)
### Inplace method (modifies the original DataFrame)
###df.drop(columns=['column1', 'column2'], inplace=True)

master_file.drop(columns=['CD_Is Subsidiary','CD_Sector','CD_Industry1'], inplace = True )
master_file.rename(columns={'CD_NSE Symbol': 'NSE_Code', 'CD_BSE Code': 'BSE_Code'}, inplace= True )


#### master_file df is Company Name	NSE_Code	BSE_Code

#print(master_file)

In [ ]:
## Merge NSE df with master file 

master_file = pd.merge(master_file,nse_merged_df_out, on= "NSE_Code", how = 'outer')
#master_file.dropna(subset=['Company Name'], inplace=True)


## merge bse df  with master file 
master_file = pd.merge(master_file,bse_merged_df_out, on ="BSE_Code", how = "outer")
#master_file.dropna(subset=['Company Name'], inplace=True)

# Drop records where both 'NSE_Code' and 'BSE_Code' are NaN or blank
master_file.dropna(subset=['NSE_Code', 'BSE_Code'], how='all', inplace=True)

# Reset the index after dropping rows
master_file.reset_index(drop=True, inplace=True)

# Drop records where both 'NSE_Delivery' and 'BSE_Delivery' are 0 or blank or NaN
master_file = master_file[~((master_file['NSE_Delivery'] == 0) & (master_file['BSE_Delivery'] == 0))]

# Reset the index after dropping rows
master_file.reset_index(drop=True, inplace=True)


# Trim "Company Name" column to a maximum length of 20 characters
master_file['Company Name'] = master_file['Company Name'].str.slice(0, 30)


# Reset the index after dropping rows
master_file.reset_index(drop=True, inplace=True)

#print(master_file)

In [ ]:

# Identify the data types of each column
column_data_types = master_file.dtypes

# Replace NaN values in object columns with empty strings ("")
object_columns = master_file.select_dtypes(include='object').columns
master_file[object_columns] = master_file[object_columns].fillna("")

# Replace NaN values in float/integer columns with 0
numeric_columns = master_file.select_dtypes(include=['float64', 'int64']).columns
master_file[numeric_columns] = master_file[numeric_columns].fillna(0)

# Print the resulting DataFrame
#print(master_file)

#master_file.info()

In [ ]:
# Define a custom function to replace Price_x with Price_y if Price_x is 0
def replace_price_x(row):
    if row['Price_x'] == 0:
        return row['Price_y']
    else:
        return row['Price_x']

    
def replace_prevclose_x(row):
    if row['PrevClose_x'] == 0:
        return row['PrevClose_y']
    else:
        return row['PrevClose_x']
    
    
def replace_segment(row):
    if row['Segment_x'] == "":
        return row['Segment_y']
    else:
        return row['Segment_x']

# Apply the custom function to the DataFrame
master_file['Price_x'] = master_file.apply(replace_price_x, axis=1)
master_file['PrevClose_x'] = master_file.apply(replace_prevclose_x, axis=1)

# Rename 'Price_x' column to 'Price'
master_file.rename(columns={'Price_x': 'Price'}, inplace=True)
master_file.rename(columns={'PrevClose_x':'PrevClose'}, inplace=True)

### segment

master_file['Segment_x'] = master_file.apply(replace_segment, axis = 1)
master_file.rename(columns={'Segment_x' :'Segment'}, inplace = True)

### add total delivery column
master_file['Total_Delivery'] = master_file['NSE_Delivery'] + master_file['BSE_Delivery']
master_file['Total_Delivery'] = master_file['Total_Delivery'].astype(int)

## add total traded value colum

master_file['Total_Traded_Value'] = ( master_file['Total_Delivery'] * master_file['Price'] ) / 100000



### reorder columns

columns_to_copy = ['Company Name','Total_Delivery', 'Price','PrevClose','Total_Traded_Value' ,'NSE_Code','BSE_Code','Segment']

# Create a new DataFrame 'final_master_file' with the desired columns
final_master_file = master_file[columns_to_copy].copy()

#### create sme file
sme_segment = ['SM','ST','M ']
sme_delivery = final_master_file[final_master_file['Segment'].isin(sme_segment)]
sme_delivery.reset_index(drop=True, inplace=True)

# Drop records where Total_Traded_Value is below 1000
final_master_file = final_master_file[final_master_file['Total_Traded_Value'] >= 1000]


In [ ]:
###creating separate file for fno delivery and non fno delivery

# Filter final_master_file to get fno_delivery by checking if NSE_Code is in fno_symbols
fno_delivery = final_master_file[final_master_file['NSE_Code'].isin(fno_symbols)]
fno_delivery_final = fno_delivery[['NSE_Code', 'Total_Delivery', 'Price','PrevClose', 'Total_Traded_Value']].copy()


# Create non_fno_delivery by dropping rows where NSE_Code is in fno_symbols
non_fno_delivery = final_master_file[~final_master_file['NSE_Code'].isin(fno_symbols)]

# Reset index inplace for fno_delivery and non_fno_delivery
fno_delivery_final.reset_index(drop=True, inplace=True)
non_fno_delivery.reset_index(drop=True, inplace=True)







In [ ]:
segment_var = ['SM','ST','M ']
final_master_file.info()
 ##Count the records in each segment
segment_counts = master_file['Segment_y'].value_counts()

print(segment_counts)

In [ ]:
### creating top 50 by value and by volume for fno and non-fno

def top_n(df,n, column_name, trade_amt):
    
    df_filtered = df[df['Total_Traded_Value'] > trade_amt]
    top_n = df_filtered.nlargest(n,column_name).sort_values(column_name, ascending = False)
    return top_n

df_value_50 = top_n(non_fno_delivery,50,'Total_Traded_Value', 250)
df_volume_50 = top_n(non_fno_delivery,50,'Total_Delivery', 250)

fno_value_50 = top_n(fno_delivery,50,'Total_Traded_Value', 1000)
fno_volume_50 = top_n(fno_delivery,50,'Total_Delivery', 1000)

fno_columns = ['NSE_Code', 'Total_Delivery','Price','PrevClose','Total_Traded_Value']

fno_value_50 = fno_value_50[fno_columns]
fno_volume_50 = fno_volume_50[fno_columns]

sme_value_50 = top_n(sme_delivery,50,'Total_Traded_Value', 100)

###########################

###########################

## write to excel -- non_fno value and volume is merged by concat and saved for multiple scrips report (7-day history)
#### sme_csv is similarly saved for 7-day history report

df_non_fno = pd.concat([df_value_50, df_volume_50])
df_non_fno_wl = df_non_fno[['NSE_Code']]

df_fno = pd.concat([fno_value_50, fno_volume_50])


# Reset the index of the series
df_non_fno_wl.reset_index(drop=True, inplace=True)


### define csv files
output_non_fno = output_path + "0_non_fno.csv"
output_non_fno_date = output_path + "0_non_fno "+nse_dely_date+".csv"
output_non_fno_wl = output_path + "0_non_fno_wl "+nse_dely_date+".csv"


output_fno = output_path + "fno.csv"

output_sme_nodate = output_path + 'sme_csv.csv'
output_sme = output_path + 'sme ' + nse_dely_date +'.csv'

### write to csv
df_non_fno.to_csv(output_non_fno, index = False)
df_non_fno.to_csv(output_non_fno_date, index = False)
df_non_fno_wl.to_csv(output_non_fno_wl, index = False)

df_fno.to_csv(output_fno, index = False)

sme_delivery.to_csv(output_sme, index=False)
sme_value_50.to_csv(output_sme_nodate)

In [ ]:





def print_formatted_rows(df, columns_to_select):
    # Iterate over the DataFrame and print each row
    for i, (_, row) in enumerate(df.iterrows(), 1):
        p_coname = row[columns_to_select[0]].ljust(40)
        p_dely = f"{row[columns_to_select[1]]:,.0f}".rjust(15)
        p_price = f"{row[columns_to_select[2]]:,.2f}".rjust(12)
        p_prevclose = f"{row[columns_to_select[3]]:,.2f}".rjust(12)
        p_trade_value = f"{row[columns_to_select[4]]:,.2f}".rjust(18)
        p_nse_code = row[columns_to_select[5]].ljust(10)
        
        print(f"{p_coname}{p_dely}{p_price}{p_prevclose}{p_trade_value}    {p_nse_code}")

        # Check if the current row index is a multiple of 5, and insert a blank line
        if i % 5 == 0:
            print()

def fno_formatted_rows(df, columns_to_select):
    
    # Iterate over the DataFrame and print each row
    for i, (_, row) in enumerate(df.iterrows(),1):
        p_nse_code = row[columns_to_select[0]].ljust(10)
        p_dely = f"{row[columns_to_select[1]]:,.0f}".rjust(15)
        p_price = f"{row[columns_to_select[2]]:,.2f}".rjust(12)
        p_prevclose = f"{row[columns_to_select[3]]:,.2f}".rjust(12)
        p_trade_value = f"{row[columns_to_select[4]]:,.2f}".rjust(12)
        
        print(f"{p_nse_code}{p_dely}{p_price}{p_prevclose}{p_trade_value}")
        
        # Check if the current row index is a multiple of 5, and insert a blank line
        if i % 5 == 0:
            print()






In [ ]:
columns_to_select = ['Company Name', 'Total_Delivery', 'Price', 'PrevClose','Total_Traded_Value', 'NSE_Code']
col_length = [40,15,12,12,18,10]
formatted_columns = [col + ' ' * (length - len(col)) for col, length in zip(columns_to_select, col_length)]
print(formatted_columns)


In [ ]:
##columns selection for printing
columns_to_select = ['Company Name', 'Total_Delivery', 'Price','PrevClose', 'Total_Traded_Value', 'NSE_Code']
col_length = [40,15,12,18,10]
fno_columns_to_select = ['NSE_Code', 'Total_Delivery', 'Price','PrevClose', 'Total_Traded_Value']



print("Top 50 deliveries by value/volume non-FnO FnO and SME ", nse_dely_date)
print("")
print("")
#print(nse_dely_date)
#print()
print("                          Top 50 delivery by value NON FNO ", nse_dely_date)
print("")
print("Company Name                                ", '  Dely_Qty','      Price','   PrevClose','           Value','   NSECode')
print("")
print_formatted_rows(df_value_50,columns_to_select)
print("")
print("")
print("")


print("                        Top 50 delivery by volume NON FNO ", nse_dely_date)
print("")
print("Company Name                                ", '  Dely_Qty','      Price','   PrevClose','           Value','   NSECode')
print("")

print_formatted_rows(df_volume_50,columns_to_select)
print("")
print("")
print("")


#### printing for fno
print("          Top 50 FNO Delivery by value ", nse_dely_date)
print("")
print("NSECode       ",'  Dely_Qty','      Price','   PrevClose','    Value' )
print("")
fno_formatted_rows(fno_value_50,fno_columns_to_select)
print("")
print("")
print("")

print("             Top 50 FNO Delivery by volume ", nse_dely_date)
print("")
print("NSECode       ",'  Dely_Qty','      Price','   PrevClose','   Value' )
print("")

fno_formatted_rows(fno_volume_50,fno_columns_to_select)
print("")
print("")
print("")


print("                          Top 50 delivery by value SME ", nse_dely_date)
print("")
print("Company Name                                ", '  Dely_Qty','      Price','   PrevClose','           Value','   NSECode')
print("")
print_formatted_rows(sme_value_50,columns_to_select)
print("")
print("")
print("")




In [ ]:
sorted_df = bse_dely_data.sort_values(by='BSE_Delivery', ascending = False)
sorted_df


In [ ]:
######### fromatting text
text = "Hello"
width = 10

left_justified_text = text.ljust(width)
print(left_justified_text)  # Output: "Hello     "

########### formatting numbers

# Example decimal value
value = 1234567.89

# Format the decimal value with 2 decimal points and commas
formatted_value = f'{value:,.2f}'

# Specify the desired width for right-justification
width = 15

# Right-justify the formatted value to the specified width
right_justified_value = formatted_value.rjust(width)

# Print the result
print(right_justified_value)

################# adding both

# Example string and number
text = "Total Traded Value:"
value = 1234567.89

# Format the number with 2 decimal points and commas
formatted_value = f'{value:,.2f}'

# Concatenate the string and the formatted number
result_string = f'{text} {formatted_value}'

# Print the result
print(result_string)


##### adding spaces
# Calculate the number of spaces needed to make the total length 50 characters
num_spaces = 30 - len(text)

# Concatenate the text with the required number of spaces and the formatted value
result_string = text + ' ' * num_spaces + formatted_value

# Print the result
print(result_string)


In [ ]:
# Value to match against
#target_name = 'Charlie'

# Locate the record(s) matching the value in the 'Name' column
#matching_records = df[df['Name'] == target_name]

#volume = 3551452
#find_nse_symbol = nse_merged_df_out[nse_merged_df_out['NSE_Delivery']== volume]
#find_nse_symbol

In [ ]:
import pandas as pd

# Example DataFrame (replace with your actual DataFrame)
data = {
    'Name': ['John', 'Alice', 'Bob'],
    'Age': [30, 25, 22],
    'City': ['New York', 'London', 'Paris']
}

df = pd.DataFrame(data)

# Add the 'Salary' column with the given values
df['Salary'] = [30000, 49304.35, 2938]

# Format the 'Salary' column with 2 decimal points and commas
df['Formatted Salary'] = df['Salary'].apply(lambda x: f'{x:,.2f}')

# Remove headers and iterate through rows using iterrows()
for _, row in df.iterrows():
    name = row['Name'].ljust(20)
    age = str(row['Age']).rjust(9)
    city = row['City'].ljust(30)
    salary = f"{row['Salary']:,.2f}".rjust(15)
    print(f"{name}{age}  {city}{salary}")
